# I’m Something of a Painter Myself

### Use GANs to create art - will you be the next Monet?

### Description

*“Every artist dips his brush in his own soul, and paints his own nature into his pictures.”*

-Henry Ward Beecher

We recognize the works of artists through their unique style, such as color choices or brush strokes. The “je ne sais quoi” of artists like Claude Monet can now be imitated with algorithms thanks to generative adversarial networks (GANs). In this getting started competition, you will bring that style to your photos or recreate the style from scratch!

Computer vision has advanced tremendously in recent years and GANs are now capable of mimicking objects in a very convincing way. But creating museum-worthy masterpieces is thought of to be, well, more art than science. So can (data) science, in the form of GANs, trick classifiers into believing you’ve created a true Monet? That’s the challenge you’ll take on!

### The Challenge:

A GAN consists of at least two neural networks: a generator model and a discriminator model. The generator is a neural network that creates the images. For our competition, you should generate images in the style of Monet. This generator is trained using a discriminator.

The two models will work against each other, with the generator trying to trick the discriminator, and the discriminator trying to accurately classify the real vs. generated images.

Your task is to build a GAN that generates 7,000 to 10,000 Monet-style images.

### Dataset Description

The dataset contains four directories: monet_tfrec, photo_tfrec, monet_jpg, and photo_jpg. The monet_tfrec and monet_jpg directories contain the same painting images, and the photo_tfrec and photo_jpg directories contain the same photos.

We recommend using TFRecords as a Getting Started competition is a great way to become more familiar with a new data format, but JPEG images have also been provided.

The monet directories contain Monet paintings. Use these images to train your model.

The photo directories contain photos. Add Monet-style to these images and submit your generated jpeg images as a zip file. Other photos outside of this dataset can be transformed but keep your submission file limited to 10,000 images.

Note: Monet-style art can be created from scratch using other GAN architectures like DCGAN. The submitted image files do not necessarily have to be transformed photos.

Check out the CycleGAN dataset to experiment with the artistic style of other artists.

### Files

monet_jpg - 300 Monet paintings sized 256x256 in JPEG format

monet_tfrec - 300 Monet paintings sized 256x256 in TFRecord format

photo_jpg - 7028 photos sized 256x256 in JPEG format

photo_tfrec - 7028 photos sized 256x256 in TFRecord format


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from math import *
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
files_monet_jpg = os.listdir("photos/monet_jpg")
files_photo_jpg = os.listdir("photos/photo_jpg")
files_monet_jpg_path = []
files_photo_jpg_path = []
for i in files_monet_jpg:
    files_monet_jpg_path.append("photos/monet_jpg/"+i)
for i in files_photo_jpg:
    files_photo_jpg_path.append("photos/photo_jpg/"+i)

In [3]:
X_train_size = int(len(files_photo_jpg_path)*0.70)
X_valid_size = int(len(files_photo_jpg_path)*0.15)
X_test_size = int(len(files_photo_jpg_path)*0.15)

y_train_size = int(len(files_monet_jpg_path)*0.70)
y_valid_size = int(len(files_monet_jpg_path)*0.15)
y_test_size = int(len(files_monet_jpg_path)*0.15)

In [4]:
train_photo_jpg = files_photo_jpg_path[0:X_train_size]
valid_photo_jpg = files_photo_jpg_path[X_train_size:X_train_size+X_valid_size]
test_photo_jpg = files_photo_jpg_path[X_train_size+X_valid_size:]

train_monet_jpg = files_monet_jpg_path[0:y_train_size]
valid_monet_jpg = files_monet_jpg_path[y_train_size:y_train_size+y_valid_size]
test_monet_jpg = files_monet_jpg_path[y_train_size+y_valid_size:]

In [5]:
img = tf.keras.preprocessing.image.load_img(path="photos/monet_jpg/0a5075d42a.jpg")

In [6]:
tens = tf.convert_to_tensor(img)

In [ ]:
X_train = []
X_valid = []
X_test = []

for i in train_photo_jpg:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    X_train.append(tensor)

for i in valid_photo_jpg:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    X_valid.append(tensor)

for i in test_photo_jpg:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    X_test.append(tensor)

X_train = np.array(X_train)/255.0
X_valid = np.array(X_valid)/255.0
X_test = np.array(X_test)/255.0

X_train.shape

In [ ]:
y_train = []
y_valid = []
y_test = []

for i in train_monet_jpg:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    y_train.append(tensor)

for i in valid_monet_jpg:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    y_valid.append(tensor)

for i in test_monet_jpg:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    y_test.append(tensor)

y_train = np.array(y_train)/255.0
y_valid = np.array(y_valid)/255.0
y_test = np.array(y_test)/255.0

y_train.shape

In [ ]:
img_tensor = tf.convert_to_tensor(img)
img_tensor.ndim

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(256,256,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
])

In [ ]:
model.summary()

In [ ]:
X_train = []
for i in files_photo_jpg_path:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    X_train.append(tensor)
X_train = np.array(X_train)/255.0

In [ ]:
y_train = []
for i in files_monet_jpg_path:
    img = tf.keras.preprocessing.image.load_img(path=i)
    tensor = tf.convert_to_tensor(img)
    y_train.append(tensor)
y_train = np.array(y_train)/255.0

In [ ]:
np.abs(np.round((X_train[0]-y_train[0]))).sum()

In [ ]:
fin1 = []
fin2 = []
maxi = []
for i in X_train:
    p = []
    q = []
    r = []
    s = []
    for j in y_train:
        p.append(np.abs(i-j))
        q.append(np.abs(i-j).sum())
        r.append(i)
        s.append(j)
    maxi = min(q)
    pos = q.index(maxi)
    fin1.append(r[pos])
    fin2.append(s[pos])

In [ ]:
fin1

In [ ]:
fin2

In [ ]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [ ]:
len(fin1)

In [ ]:
len(fin2)

In [ ]:
fin1[0]

In [ ]:
tf.keras.preprocessing.image.array_to_img(fin1[0])

In [ ]:
tf.keras.preprocessing.image.array_to_img(fin2[0])